<a href="https://colab.research.google.com/github/njsteiger/njsteiger.github.io/blob/main/gws_assignment_9_GCM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Assignment 9: Global Climate Modeling

In this assignment we will gain experience running a climate model called [SpeedyWeather](https://speedyweather.github.io/SpeedyWeatherDocumentation/stable/) and analyzing the results. The climate model is written in the computing language of Julia. And if you're curious, the version of the model we will be using solves [these sets of differential equations](https://speedyweather.github.io/SpeedyWeatherDocumentation/stable/primitiveequation/).

To do this you will notice that we are using a document that incorporates both text and computer code, specifically a Jupyter notebook. We are going to run the simulations using Google's [Colab](https://colab.google/) cloud computing so that we don't have to have everyone install Julia on their own computer and we can do everything through an internet browser.

### Loading Packages

Packages are the parts of a computing language that allow us run the climate model and analyze the model output. Note that it can take several minutes for the packages to all load.

In [ ]:
import Pkg
Pkg.add("Statistics")
Pkg.add("SpeedyWeather")
Pkg.add("NCDatasets")
Pkg.add("Plots")

In [ ]:
using Statistics, Plots, SpeedyWeather, NCDatasets

## Model setup and running the climate model

For this first exercise we're going to set up the climate model and learn how to run it. The first step is that we need to tell the climate model, named SpeedyWeather, what horizontal and vertical resolution we want. We can have the model run at very low or very high resolution.

In the code below there are two options tha control the resolution: 'trunc' (short for 'truncation') controls the horizontal resultion while 'nlayers' controls the number of vertical layers of the model. Visually this is represented by the following figure from Chapter 10 of the class textbook. Note the horizontal and vertical resoltions are not necessarily the same physical size (grid boxes are not perfect cubes); in fact, climate model grids are usually much wider than they are tall.

<img src="https://njsteiger.github.io/gws/img/model-grid-neelin.png" width="600">

For the initial resolution, 'trunc' is set to 20 and 'nlayers' is set to 6.

In [ ]:
# Define the climate model resolution (truncation and layers)
spectral_grid = SpectralGrid(trunc=20, nlayers=6)

This section of code sets up the climate model.

In [ ]:
# Construct the climate model type (PrimitiveWetModel)
model = PrimitiveWetModel(spectral_grid)

# Initialize all model components
simulation = initialize!(model)

Now we will run it! Below we've set it to run for 10 days. So the climate model is simulating 20 days of the weather/climate. If you wanted to run a climate model for real research questions then you often need to run the model for many model years, either decades to centuries, unless you are studying fast processes like the behavior of clouds.

In [ ]:
# Run the climate model
run!(simulation, period=Day(10), output=true)

Once the model has run we have output variables that are climate variables such as wind, specific humidity, temperature, etc. The model output was saved in a format called NetCDF, which is the standard format used in all kinds of climate data. I've named the output to a particular variable called 'ds'. You can see below a list of all the variables that were saved to 'ds'. One thing to note is that we ran the simulation for 10 days but that is not the number that we see as 'time'; this variable is just the periods in time that we saved the data over (default is every 3 hours).

In [ ]:
run_folder = model.output.run_folder # Find where the model output was saved
ds = NCDataset("$run_folder/output.nc") # Save the model output to variable 'ds'

Now we can finally plot some of the output variables. Below is a plot of global temperature in the layer of the atmosphere that is closest to the surface and for the last time step that we saved the data for. This is indicated in the code `ds["temp"][:,:,6,end]`, which is saying to select the temperature variable (named `temp`) and to select all the longitude locations and latitude locations (`:`), and then to select the 6th of the 6 layers (`6`) and the last time step (`end`).

In [ ]:
# Temperature plot
heatmap(ds["lon"][:], reverse(ds["lat"][:]), reverse(ds["temp"][:,:,6,end]',dims=1),
xlabel="Longitude", ylabel="Latitude", title="Temperature [°C]")

Below is a similar plot but with the variable relative vorticity (called `vor`, which we saw in the list of output variables above). This variable shows how much the air is rotating (rotation is usually high where there are storms). Notice that only the name of the variable has changed from above.

In [ ]:
# Vorticity plot
heatmap(ds["lon"][:], reverse(ds["lat"][:]), reverse(ds["vor"][:,:,6,end]',dims=1),
xlabel="Longitude", ylabel="Latitude", title="Relative Vorticity [1/s]")

And now this is a plot for specific humidity, `humid`.

In [ ]:
# Humidity plot
heatmap(ds["lon"][:], reverse(ds["lat"][:]), reverse(ds["humid"][:,:,6,end]',dims=1),
xlabel="Longitude", ylabel="Latitude", title="Specific Humidity [kg/kg]")

## Exercise 1

👉 Make plots of temperature, voriticity, and humidity, just like we did above (copy and paste the code into new cells below) but change the atmospheric layer so that we are plotting a middle layer of the atmosphere. Compare and contrast the similarities and differences that you see in the two layers and between the three variables (e.g. how are temperature and humidity related at individual levels). Give possible physical explanations for these differences and similarities.

In [ ]:
# Write your explanation here

## Exercise 2

👉 Re-run SpeedyWeather but change the model to be higher resolution, specifically a truncation of 63 and 11 layers. Be sure to rename each of the output variables so that you don't overwrite the previous ones (e.g., so after copy-pasting the line for `spectral_grid`, you should change it in your high resolution version below to something like `spectral_grid_hi`)

👉 Now make the same plots as you did before for temperature, specific humidity, and relative voriticity, for a middle atmospheric layer, but now for your high resolution simulation. Explain what is different about the results between the high and low resolution simulations.

In [ ]:
# Write your explanation here

## Exercise 3

Now we're going to make animations of the data from the higher resolution. To do this we'll use the `@animate` macro and make plots of every time step by looping through each time step. Below is an incomplete outline of code that you can use to create animations. You will need to modify variable names so that they correspond to your code to get it to run properly.

👉 Make animations for humidity and relative vorticity, for a middle atmospheric layer, for all the time steps. Explain what is happening here at different time steps, particularly at the very beginning and then as it progresses through the simulation. Use experience you've gained from running previous models that you have used in the class to explain any un-physical features that you might see.

In [ ]:
# Create an animation from the humidity data
anim_var1 = @animate for i=1:number_of_time_steps
    heatmap(lon, lat, data[:,:,middle_atmos_layer,i],
            title="Time step $i",
            clim=(mimimum_plot_value, maximum_plot_value)) # Use fixed limits for stable colorbars
end

# Save plots as a movie
mp4(anim_var1, "movie_var1.mp4", fps = 1)